In [ ]:
!pip install transformers==2.5.1
!pip install torch==1.5.0+cu92 torchvision==0.6.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install git+https://github.com/huggingface/transformers


In [ ]:
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/question-answering/run_squad.py

In [ ]:
%env DATA_DIR=./data/squad 

# download the data
def download_squad():
        !wget -P $DATA_DIR https://github.com/TQuad/turkish-nlp-qa-dataset/blob/master/dev-v0.1.json
        !wget -P $DATA_DIR https://github.com/TQuad/turkish-nlp-qa-dataset/blob/master/train-v0.1.json
            
download_squad()

In [ ]:
cmd = [
    'python', 
#    '-m torch.distributed.launch --nproc_per_node 2', # use this to perform distributed training over multiple GPUs
    'run_squad.py', 
    
    '--model_type', 'bert',                            # model type (one of the list under "Pick a Model" above)
    
    '--model_name_or_path', 'bert-base-tr-qa-v1',       # specific model name of the given model type (shown, a list is here: https://huggingface.co/transformers/pretrained_models.html) 
                                                       # on first execution this initiates a download of pre-trained model weights;
                                                       # can also be a local path to a directory with model weights
    
    '--output_dir', './models/bert/bbu_squad2',        # directory for model checkpoints and predictions
    
#    '--overwrite_output_dir',                         # use when adding output to a directory that is non-empty --
                                                       # for instance, when training crashes midway through and you need to restart it
    
    '--do_train',                                      # execute the training method 
    
    '--train_file', '$DATA_DIR/train-v0.1.json',       # provide the training data
    
    '--version_2_with_negative',                       # ** MUST use this flag if training on SQuAD 2.0! DO NOT use if training on SQuAD 1.1
    
    '--do_lower_case',                                 # ** set this flag if using an uncased model; don't use for Cased Models
    
    '--do_eval',                                       # execute the evaluation method on the dev set -- note: 
                                                       # if coupled with --do_train, evaluation runs after fine-tuning 
    
    '--predict_file', '$DATA_DIR/dev-v0.1.json',       # provide evaluation data (dev set)
    
    '--eval_all_checkpoints',                          # evaluate the model on the dev set at each checkpoint
    
    '--per_gpu_eval_batch_size', '12',                 # evaluation batch size for each gpu
    
    '--per_gpu_train_batch_size', '12',                # training batch size for each gpu
    
    '--save_steps', '5000',                            # how often checkpoints (complete model snapshot) are saved 
    
    '--threads', '8',                                  # num of CPU threads to use for converting SQuAD examples to model features
    
    # --- Model and Feature Hyperparameters --- 
    '--num_train_epochs', '2',                         # number of training epochs - usually 2-3 for SQuAD 
    
    '--learning_rate', '3e-5',                         # learning rate for the default optimizer (Adam in this case)
    
    '--max_seq_length', '512',                         # maximum length allowed for the full input sequence 
    
    '--doc_stride', '128'                              # used for long documents that must be chunked into multiple features -- 
       ]

In [ ]:
def download_squad(version=1):
    if version == 1:
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
    else:
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
            
download_squad(version=1)

In [ ]:
!python run_squad.py  \
    --model_type bert   \
    --model_name_or_path dbmdz/bert-base-turkish-cased  \
    --output_dir models/bert/ \
    --data_dir data/squad   \
    --overwrite_output_dir \
    --overwrite_cache \
    --do_train  \
    --train_file train-v0.1-cv.json   \
    --version_2_with_negative \
    --do_eval   \
    --predict_file dev-v0.1-cv.json   \
    --per_gpu_train_batch_size 16   \
    --learning_rate 3e-5   \
    --num_train_epochs 2.0   \
    --max_seq_length 512   \
    --doc_stride 128   \
    --threads 10   \
    --save_steps 5000 

In [19]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("./models/bert/")
model = AutoModelForQuestionAnswering.from_pretrained("./models/bert/")

In [27]:
question = "I.Murat'ın annesi kimdir?"

context = """I.Murat 1326 yılında Bursada doğmuştur. I.Murat 1389 yılında Kosavada ölmüştür.
    Osmanlı İmparatorluğu'nun üçüncü padişahı olan I.Murat, 1359 ile 1389 yılları arasında beylik yapmıştır.
    I.Murat'ın babası Orhan Gazi, annesi Nilüfer Hatun'dur.
    I.Murat babası Orhan Gazi döneminde 95.000 kilometrekare olarak almış, devlet topraklarını I.Murat döneminde topraklarını yaklaşık 500.000 kilometrekareye genişlemiştir.
    1362 yılında Ankara Eratna beyliğinden yeniden alınmıştır.
    1362 yılında Sazlıdere savaşı ile gerçekleşmiştir. Sazlıdere savaşı sonucunda Edirne ve Filibe alınmıştır. Sazlıdere savaşı Osmanlı devleti ile Bizans ve Bulgar güçleri arasında gerçekleşmiştir.
    Hamitoğullarından Eğridir ve çevresi satın alındı.
    1364 yılında Sırpsındığı savaşı gerçekleşti. Balkan devletlerinden oluşan haçlı ordusunun başında Sırp Kralı I.Layoş vardı.
    Haçlı ordusu, Hacı İlbeyi tarafından yapılan ani bir baskın ile yok edilmiştir.
    Sırpsındığı savaşından sonra Edirne başkent yapılmıştır.
    Sırpsındığı savaşı Osmanlıların Balkanlarda haçlılarla yaptığı ilk savaştır.
    Bulgar krallığı Sırpsındığı savaşı sonrası Osmanlı Devletine bağlanmıştır.
    1371 yılında Çirmen savaşı gerçekleşti. Çirmen savaşı Evranos Bey ile Sırplar arasında olmuştur. Çirmen Savaşını Osmanlı Devleti kazanmıştır.
    Çirmen savaşı sonucunda Makedonya'nın bir kısmı alındı."""


 
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 


answer_start_scores, answer_end_scores = model(**inputs)
answer_start = torch.argmax(answer_start_scores)  
answer_end = torch.argmax(answer_end_scores) + 1  


tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'Nilüfer Hatun'